In [1]:
import pickle
import re

In [2]:
repertoire_racine = '/home/michel/jorf_plat/'
with open(repertoire_racine + 'liste_paragraphe.txt', 'rb') as f:
    liste_paragraphe = pickle.load(f)

## Sépare en mots

In [54]:
mots_pp = [p.split() for p in liste_paragraphe]  # pp = par paragraphe

In [59]:
# caractères de ponctuation aux extrémités
liste_ponctuation = ['.', ',', ';', ':', '!', '?', "'", '"', '(', ')', '[', ']', '«', '»']

nouveau_mots_pp = []
nb_modification_pp = []
for i_mots, mots in enumerate(mots_pp):
    
    modification = True
    nb_modification = 0
    while modification:
        modification = False

        nouveau_mots = []
        for mot in mots:
            if mot[0] in liste_ponctuation:
                nouveau_mots.append(mot[0])
                mot = mot[1:]
            if not mot:
                continue
            a_ajouter = ''
            if mot[-1] in liste_ponctuation:
                a_ajouter = mot[-1]
                mot = mot[:-1]
            if mot:
                nouveau_mots.append(mot)
            if a_ajouter:
                nouveau_mots.append(a_ajouter)
            
        if len(nouveau_mots) > len(mots):
            modification = True
        mots = nouveau_mots
        nb_modification += 1    
            
    nouveau_mots_pp.append(mots)
    nb_modification_pp.append(nb_modification)
    
    if i_mots % 1000000 == 0:
        print(i_mots)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000


In [81]:
mots_pp = nouveau_mots_pp

In [82]:
with open(repertoire_racine + 'mots_pp_1.txt', 'wb') as f:
    pickle.dump(mots_pp, f)

In [83]:
# apostrophe
liste_apostrophe = ["'", '’']
nouveau_mots_pp = []
for mots in mots_pp:
    nouveau_mots = []
    for mot in mots:
        for apostrophe in liste_apostrophe[1:]:
            mot = mot.replace(apostrophe, "'")
        nouveau_mots += mot.split(liste_apostrophe[0])
    nouveau_mots_pp.append(nouveau_mots)
mots_pp = nouveau_mots_pp

In [84]:
with open(repertoire_racine + 'mots_pp_2.txt', 'wb') as f:
    pickle.dump(mots_pp, f)

In [3]:
with open(repertoire_racine + 'mots_pp_2.txt', 'rb') as f:
    mots_pp = pickle.load(f)

## Repère les numéros (dates, nombres, chiffres romains...)
Ceci permet de réduire la taille du vocabulaire en regroupant les nombres de même type.

### Dates

In [92]:
jeton_date = '__date'

def est_entier(v):
    for c in v:
        if c not in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
            return False
    if not v:
        return False
    if v[0] == '0':
        return False
    return True

nouveau_mots_pp = []
for i_mots, mots in enumerate(mots_pp):
    nouveau_mots = []
    i = 0
    while i < len(mots):
        est_date = False
        
        if i <= len(mots) - 3:
            mot_jour = mots[i]
            if est_entier(mot_jour) or mot_jour == '1er':
                if mot_jour == '1er':
                    num_jour = 1
                else:
                    num_jour = int(mot_jour)
                if num_jour >= 1 and num_jour <= 31:
                    mot_mois = mots[i+1]
                    if mot_mois in ['janvier', 'février', 'mars', 'avril',
                                    'mai', 'juin', 'juillet', 'août',
                                    'septembre', 'octobre', 'novembre', 'décembre']:
                        mot_annee = mots[i+2]
                        if est_entier(mot_annee):
                            num_annee = int(mot_annee)
                            if num_annee >= 1000 and num_annee <= 2500:
                                est_date = True
                            
        if est_date:
            nouveau_mots.append(jeton_date)
            i += 3
        else:
            nouveau_mots.append(mots[i])
            i += 1
            
    if i_mots % 1000000 == 0:
        print(i_mots)

    nouveau_mots_pp.append(nouveau_mots)


0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000


In [94]:
mots_pp = nouveau_mots_pp

In [95]:
with open(repertoire_racine + 'mots_pp_3.txt', 'wb') as f:
    pickle.dump(mots_pp, f)

### Chiffres arabes

In [99]:
jeton_arabe = '__arabe'

def est_entier(v):
    for c in v:
        if c not in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
            return False
    if not v:
        return False
    if v[0] == '0':
        return False
    return True

nouveau_mots_pp = []
for i_mots, mots in enumerate(mots_pp):
    nouveau_mots = []
    for mot in mots:
        if est_entier(mot):
            nouveau_mots.append(jeton_arabe)
        else:
            nouveau_mots.append(mot)
            
    if i_mots % 1000000 == 0:
        print(i_mots)

    nouveau_mots_pp.append(nouveau_mots)


0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000


In [100]:
mots_pp = nouveau_mots_pp

In [101]:
with open(repertoire_racine + 'mots_pp_4.txt', 'wb') as f:
    pickle.dump(mots_pp, f)

### Chiffres romains

In [106]:
regex_romain_minuscule = r'^(x{0,3})(ix|iv|v?i{0,4})$'
regex_romain_majuscule = r'^(X{0,3})(IX|IV|V?I{0,4})$'
jeton_romain_minuscule = '__romain_min'
jeton_romain_majuscule = '__romain_maj'

nouveau_mots_pp = []
for i_mots, mots in enumerate(mots_pp):
    nouveau_mots = []
    for mot in mots:
        if re.match(regex_romain_minuscule, mot):
            nouveau_mots.append(jeton_romain_minuscule)
        elif re.match(regex_romain_majuscule, mot):
            nouveau_mots.append(jeton_romain_majuscule)
        else:
            nouveau_mots.append(mot)
            
    if i_mots % 1000000 == 0:
        print(i_mots)

    nouveau_mots_pp.append(nouveau_mots)


0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000


In [107]:
mots_pp = nouveau_mots_pp

In [108]:
with open(repertoire_racine + 'mots_pp_5.txt', 'wb') as f:
    pickle.dump(mots_pp, f)

### Nombres arabes composés (ex: loi 2008-243, article L. 1223-6)

In [111]:
regex_arabe_comp = r'^\d+-\d+$'
jeton_arabe_comp = '__arabe_comp'

nouveau_mots_pp = []
for i_mots, mots in enumerate(mots_pp):
    nouveau_mots = []
    for mot in mots:
        if re.match(regex_arabe_comp, mot):
            nouveau_mots.append(jeton_arabe_comp)
        else:
            nouveau_mots.append(mot)
            
    if i_mots % 1000000 == 0:
        print(i_mots)

    nouveau_mots_pp.append(nouveau_mots)


0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000


In [112]:
mots_pp = nouveau_mots_pp

In [113]:
with open(repertoire_racine + 'mots_pp_6.txt', 'wb') as f:
    pickle.dump(mots_pp, f)

### Numérotation code (ex: L. 1223-6, R. 1223-7, L1223-6, R1223-6)
Pour l'instant, la manière longue (X. DDD-DDD) n'est pas traitée (déjà __arabe_comp pour réduire la cardinalité)

In [116]:
regex_num_article = r'^(L|R)\d+-\d+$'
jeton_num_article = '__num_article'

nouveau_mots_pp = []
for i_mots, mots in enumerate(mots_pp):
    nouveau_mots = []
    for mot in mots:
        if re.match(regex_num_article, mot):
            nouveau_mots.append(jeton_num_article)
        else:
            nouveau_mots.append(mot)
            
    if i_mots % 1000000 == 0:
        print(i_mots)

    nouveau_mots_pp.append(nouveau_mots)


0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000


In [118]:
mots_pp = nouveau_mots_pp

In [119]:
with open(repertoire_racine + 'mots_pp_7.txt', 'wb') as f:
    pickle.dump(mots_pp, f)

###  Adverbes multiplicatifs latins (bit, ter...)

In [123]:
latins = [
    'bis',
    'ter',
    'quater',
    'quinquies',
    'sexies',
    'septies',
    'octies',
    'nonies',
    'decies',
    'undecies',
    'duodecies',
    'terdecies',
    'quaterdecies',
    'quindecies',
    'sedecies',
    'sexdecies',
    'septdecies',
    'duodevicies',
    'octodecies',
    'undevicies',
    'novodecies',
    'vicies',
]

jeton_latin = '__latin'

nouveau_mots_pp = []
for i_mots, mots in enumerate(mots_pp):
    nouveau_mots = []
    for mot in mots:
        if mot in latins:
            nouveau_mots.append(jeton_latin)
        else:
            nouveau_mots.append(mot)
            
    if i_mots % 1000000 == 0:
        print(i_mots)

    nouveau_mots_pp.append(nouveau_mots)


0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000


In [124]:
mots_pp = nouveau_mots_pp

In [125]:
with open(repertoire_racine + 'mots_pp_8.txt', 'wb') as f:
    pickle.dump(mots_pp, f)

### Ordinaux ° (ex: 2°)

In [139]:
regex_ordinal_o = r'^\d+°$'
jeton_ordinal_o = '__ordinal_o'

nouveau_mots_pp = []
for i_mots, mots in enumerate(mots_pp):
    nouveau_mots = []
    for mot in mots:
        if re.match(regex_ordinal_o, mot):
            nouveau_mots.append(jeton_ordinal_o)
        else:
            nouveau_mots.append(mot)
            
    if i_mots % 1000000 == 0:
        print(i_mots)

    nouveau_mots_pp.append(nouveau_mots)


0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000


In [140]:
mots_pp = nouveau_mots_pp

In [141]:
with open(repertoire_racine + 'mots_pp_9.txt', 'wb') as f:
    pickle.dump(mots_pp, f)